In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [2]:
import pandas as pd
df = pd.read_excel('wingsfood_1000_data.xlsx')

In [3]:
df.head()

,Nama Produk,Qty,Harga Satuan,Subtotal
0,Pop Mie Kari Ayam Cup,2,5800,11600
1,Ale-Ale Botol 250ml,5,4500,22500
2,Top Coffee Sachet 10x20g,1,11500,11500
3,Pop Mie Ayam Bawang Cup,3,5800,17400
4,Floridina Jeruk Botol 350ml,4,5200,20800


In [4]:
df = df.dropna(subset=["Nama Produk", "Harga Satuan"])

In [5]:
hasil_ocr = ['POP MIE AYAM 75G']  

In [6]:
standart = df["Nama Produk"].astype(str).tolist()
gabungan = standart + hasil_ocr

In [7]:
vector = TfidfVectorizer()
tfidf= vector.fit_transform(gabungan)

standart_vec = tfidf[:len(standart)]
ocr_vec = tfidf[len(standart):]


In [8]:
kemiripan = cosine_similarity(ocr_vec, standart_vec)

In [9]:
threshold = 0.3

for i, row in enumerate(kemiripan):
    idx_paling_mirip = np.argmax(row)
    skor_tertinggi = row[idx_paling_mirip]
    
    if skor_tertinggi >= threshold:
        deskripsi_cocok = standart[idx_paling_mirip]
        harga_jual = df.iloc[idx_paling_mirip]["Harga Satuan"]
        harga_jual = round(harga_jual)
        print(f"OCR: '{hasil_ocr[i]}'")
        print(f"→ sesuai dengan: '{deskripsi_cocok}' (Skor: {skor_tertinggi:.2f})")
        print(f"→ Harga Jual: {harga_jual}\n")
    else:
        print(f"OCR: '{hasil_ocr[i]}'")
        print(f"→ Tidak ditemukan barang yang cocok (Skor tertinggi: {skor_tertinggi:.2f})\n")


OCR: 'POP MIE AYAM 75G'
→ sesuai dengan: 'Pop Mie Ayam Bawang Cup' (Skor: 0.37)
→ Harga Jual: 5800



In [ ]:
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image
import pytesseract
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

df = pd.read_excel('SKUNivea.xlsx', sheet_name='All NIVEA PL Mei 2024', skiprows=2,usecols=range(9) )  # pastikan file ada di folder yang sama
df = df.dropna(subset=["Description", "Harga Jual ke Konsumen yg Disarankan"])
standart = df['Description'].astype(str).tolist()

def cari_deskripsi_mirip(nama_ocr):
    gabungan = standart + [nama_ocr]
    vectorizer = TfidfVectorizer().fit(gabungan)
    tfidf = vectorizer.transform(gabungan)
    
    standart_vec = tfidf[:len(standart)]
    ocr_vec = tfidf[len(standart):]

    similarity = cosine_similarity(ocr_vec, standart_vec)
    skor = np.max(similarity)
    if skor < 0.3:  
        return "Barang Tidak Ditemukan", 0
    idx = np.argmax(similarity)
    deskripsi_cocok = standart[idx]
    harga = df.iloc[idx]["Harga Jual ke Konsumen yg Disarankan"]
    return deskripsi_cocok, round(harga)

pytesseract.pytesseract.tesseract_cmd=r"D:\ocr\tesseract.exe"
def process_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
    if not file_path:
        return

    text = pytesseract.image_to_string(Image.open(file_path))
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    if not lines:
        return

    nama_toko = lines[0]

    pemisah_patterns = [
        r"[-=_]{5,}",
        r"\b(HARGA JUAL|TOTAL|TUNAI|KEMBALI|ANDA HEMAT|DPP|PPN|LAYANAN KONSUMEN)\b"
    ]

    barang_lines = []
    for line in lines:
        if any(re.search(pat, line.upper()) for pat in pemisah_patterns):
            break
        if re.search(r'\d+\s+[\d.,]+\s+[\d.,]+$', line):
            barang_lines.append(line)

    data = []
    for line in barang_lines:
        clean = line.replace(",", "").replace(".", "")
        parts = clean.split()
        if len(parts) >= 4:
            qty = parts[-3]
            satuan = parts[-2]
            nama_barang = " ".join(parts[:-3])
            deskripsi_cocok, harga_jual = cari_deskripsi_mirip(nama_barang)
            total = float(harga_jual) * int(qty)
            data.append([nama_toko, nama_barang, deskripsi_cocok, qty, harga_jual, total])

    for i in table.get_children():
        table.delete(i)

    for row in data:
        table.insert("", tk.END, values=row)
root = tk.Tk()
root.title("OCR Struk Belanja")
root.geometry("1000x600")

frame = tk.Frame(root)
frame.pack(pady=10)

btn = tk.Button(frame, text="Upload Gambar Struk", command=process_image)
btn.pack()

cols = ("Nama Toko", "Nama Barang", "Cocok Deskripsi", "Qty", "Harga Satuan", "Harga Total")
table = ttk.Treeview(root, columns=cols, show="headings", height=15)
for col in cols:
    table.heading(col, text=col)
    table.column(col, anchor="center", width=160)

table.pack(pady=10, fill="both", expand=True)

root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image
import pytesseract
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

df = pd.read_excel('SKUNivea.xlsx', sheet_name='All NIVEA PL Mei 2024', skiprows=2, usecols=range(9))
df = df.dropna(subset=["Description", "Harga Jual ke Konsumen yg Disarankan"])
standart = df['Description'].astype(str).tolist()
def cari_deskripsi_mirip(nama_ocr):
    gabungan = standart + [nama_ocr]
    vectorizer = TfidfVectorizer().fit(gabungan)
    tfidf = vectorizer.transform(gabungan)
    
    standart_vec = tfidf[:len(standart)]
    ocr_vec = tfidf[len(standart):]

    similarity = cosine_similarity(ocr_vec, standart_vec)
    skor = np.max(similarity)
    
    if skor < 0.3:
        print(f"[WARNING] Skor terlalu rendah ({skor:.2f}) untuk '{nama_ocr}'. Barang tidak ditemukan.")
        return "Barang Tidak Ditemukan", 0
    
    idx = np.argmax(similarity)
    deskripsi_cocok = standart[idx]
    harga = df.iloc[idx]["Harga Jual ke Konsumen yg Disarankan"]
    print(f"[INFO] '{nama_ocr}' cocok dengan '{deskripsi_cocok}' (Skor: {skor:.2f}) → Harga: {harga}")
    return deskripsi_cocok, round(harga)

pytesseract.pytesseract.tesseract_cmd = r"D:\ocr\tesseract.exe"

def process_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
    if not file_path:
        print("[ERROR] Tidak ada file yang dipilih.")
        return

    try:
        image = Image.open(file_path)
        print(f"[INFO] Gambar berhasil dibuka: {file_path}")
    except Exception as e:
        print(f"[ERROR] Gagal membuka gambar: {e}")
        return

    try:
        text = pytesseract.image_to_string(image)
        print("[INFO] OCR berhasil. Hasil teks:")
        print(text)
    except Exception as e:
        print(f"[ERROR] OCR gagal: {e}")
        return

    lines = [line.strip() for line in text.splitlines() if line.strip()]
    if not lines:
        print("[WARNING] Tidak ada teks yang terdeteksi dari OCR.")
        return

    nama_toko = lines[0]
    print(f"[INFO] Nama toko terdeteksi: {nama_toko}")

    pemisah_patterns = [
        r"[-=_]{5,}",
        r"\b(HARGA JUAL|TOTAL|TUNAI|KEMBALI|ANDA HEMAT|DPP|PPN|LAYANAN KONSUMEN)\b"
    ]

    barang_lines = []
    for line in lines:
        if any(re.search(pat, line.upper()) for pat in pemisah_patterns):
            break
        if re.search(r'\d+\s+[\d.,]+\s+[\d.,]+$', line):
            barang_lines.append(line)

    print(f"[INFO] Baris barang terdeteksi: {barang_lines}")

    data = []
    for line in barang_lines:
        clean = line.replace(",", "").replace(".", "")
        parts = clean.split()
        if len(parts) >= 4:
            try:
                qty = int(parts[-3])
                satuan = float(parts[-2])
                nama_barang = " ".join(parts[:-3])
                deskripsi_cocok, harga_jual = cari_deskripsi_mirip(nama_barang)
                total = harga_jual * qty
                data.append([nama_toko, nama_barang, deskripsi_cocok, qty, harga_jual, total])
            except Exception as e:
                print(f"[ERROR] Gagal memproses baris '{line}': {e}")
        else:
            print(f"[WARNING] Format baris tidak sesuai: '{line}'")

    for i in table.get_children():
        table.delete(i)

    for row in data:
        table.insert("", tk.END, values=row)

root = tk.Tk()
root.title("OCR Struk Belanja")
root.geometry("1000x600")

frame = tk.Frame(root)
frame.pack(pady=10)

btn = tk.Button(frame, text="Upload Gambar Struk", command=process_image)
btn.pack()

cols = ("Nama Toko", "Nama Barang", "Cocok Deskripsi", "Qty", "Harga Satuan", "Harga Total")
table = ttk.Treeview(root, columns=cols, show="headings", height=15)
for col in cols:
    table.heading(col, text=col)
    table.column(col, anchor="center", width=160)

table.pack(pady=10, fill="both", expand=True)

root.mainloop()


[INFO] Gambar berhasil dibuka: D:/pkl/1.1/qq.png
[INFO] OCR berhasil. Hasil teks:
PT. INDOMARCO PRISMATAMA

JL. ANCOL I/9-10, ANCOL

BARAT- JAKARTA UTARA

NPWP : 01.337.994.6-292.000

ONLINE STORE - NO. INVOICE : 01230706001
01.07.25 - 15:43 2.3.17 382765/ONLINE/@1
NIVEA CRM CARE SHUR 2 21.508 43.008
NIVEA MEN CK ROLL 1 18.908 © 18.988
NIVEA LP PCH SHN 3 14.508 43.500
NIVEA SUN FC SRM 50+ 1 57.008 © 57.008
NIVEA EXBR NGT BL 1 36.808 © 36.800
NIVEA MEN DP SPY 2 29.508 59.008
NIVEA LUM 630 TRT 1 135.008 135.000
NIVEA PRL BTY ROLL 2 17.808 © 35.600
NIVEA FC CLN MLK 1 23.908 ©=—-23.988
NIVEA BBY MLD SHP 2 19.008 38.908
HARGA JUAL 498.700
ONGKOS KIRIM 9.508
TOTAL BAYAR 500.208
TERIMA KASTH

BELANJA ONLINE DI INDOMARET|

[INFO] Nama toko terdeteksi: PT. INDOMARCO PRISMATAMA
[INFO] Baris barang terdeteksi: ['NIVEA CRM CARE SHUR 2 21.508 43.008', 'NIVEA LP PCH SHN 3 14.508 43.500', 'NIVEA MEN DP SPY 2 29.508 59.008', 'NIVEA LUM 630 TRT 1 135.008 135.000', 'NIVEA BBY MLD SHP 2 19.008 38.908']
[

In [ ]:
import albumentations as A
from PIL import Image
import numpy as np

image = np.array(Image.open('struk.jpg'))

transform = A.Compose([
    A.Rotate(limit=10, p=1.0), 
    A.RandomBrightnessContrast(p=1.0),
    A.CLAHE(p=0.5),  
    A.Resize(800, 1000),  
])

augmented = transform(image=image)
processed_image = augmented["image"]

Image.fromarray(processed_image).save("processed_struk.jpg")
